In [151]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import mne 
from importlib import reload
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
# import own functions
from utils import find_folders
import dat_preproc
import seaborn as sns

In [152]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


In [154]:
subID = 'Sub045'

raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'with_med_FTG',
        subID + '_FIF.fif'
    )
)

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\with_med_FTG\Sub045_FIF.fif...
    Reading extended channel information
    Range : 0 ... 145611 =      0.000 ...   582.444 secs
Ready.


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_12144\4073594750.py:3: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\with_med_FTG\Sub045_FIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join(


In [165]:
reload(dat_preproc)
%matplotlib qt
matplotlib.rc('font', size=10)
x = raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

In [167]:
ps_allamps = []

side = 0


timepoints = [1,	103,	122,	140,	163,	550]
amplitudes = ['preDBS', 'mA_0.5', 'mA_1', 'mA_1.5', 'mA_2', 'postDBS']

for i, tp in enumerate(timepoints):
    ps = np.mean(Sxx[side,:,tp:tp+10],1)
    plt.plot(ps, label = amplitudes[i])
    ps_allamps.append(ps)


plt.xlim(40,100)
plt.ylim(0,0.4)
plt.legend()
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power')

Text(0, 0.5, 'LFP Power')

In [162]:
all_ampsDF = pd.DataFrame(np.transpose(ps_allamps))
all_ampsDF.columns = amplitudes
print(all_ampsDF)

       preDBS    mA_0.5       mA_1     mA_1.5       mA_2   postDBS
0    1.079491  1.458150   0.709699   1.173480   1.924483  0.593368
1    5.583388  3.860294   5.832766   6.618730   5.614416  2.306109
2    4.380720  1.520412   5.603415   5.735847   6.300453  2.116370
3    2.420480  1.655554   3.462885   2.916900   3.413809  1.324942
4    1.138399  2.728473   1.831148   1.948065   2.028639  1.342228
..        ...       ...        ...        ...        ...       ...
121  0.000295  0.000221   0.000432   0.000621   0.000302  0.000313
122  0.000305  0.000292   0.000595   0.000277   0.000188  0.000256
123  0.000150  0.000342   0.000800   0.000487   0.000513  0.000188
124  0.138325  4.326760   5.815009   6.719473  15.264565  0.000215
125  0.275592  8.659738  11.606733  13.452119  30.542452  0.000088

[126 rows x 6 columns]


In [163]:
all_ampsDF.to_excel(os.path.join(
    project_path,
   'data', 'power_spectra', 'with_med_FTG',
   subID + '_All_Amps_PS.xlsx'
), index = False
)

In [164]:
plt.savefig(os.path.join(
    project_path,
    'figures', 'Power_Spectra','with_med_FTG',
    subID + '_All_Amps_PS'
), dpi = 200
)

In [159]:
side1 = 0

tp = 103
ps = np.mean(Sxx[side1,:,tp:tp+10],1)
plt.plot(ps, label = amplitudes[i])
plt.xlim(40,100)
plt.ylim(0,0.2)

(0.0, 0.2)

In [188]:
peak_freqs = pd.read_excel(os.path.join(
    project_path,
    'results',
    'PeakFreqs_df.xlsx'
))
print(peak_freqs)

    SubID  preDBS  preSub  Sub  PostDBS
0  Sub005      77    75.0   65       76
1  Sub006      81    82.0   65       84
2  Sub009      73    73.0   65       73
3  Sub014      73    71.0   65       73
4  Sub017      76    78.0   63       75
5  Sub029      83    71.0   63       84
6  Sub045      84     NaN   63       84


In [198]:
# Reshape the dataframe to have a single column for the boxplot values
df_boxplot = pd.melt(peak_freqs, id_vars=['SubID'], value_vars=['preDBS', 'preSub', 'Sub', 'PostDBS'],
                     var_name='Condition', value_name='Value')

# Plot the boxplots with individual points
grouped_values = df_boxplot.groupby('Condition')['Value'].apply(list)

fig, ax = plt.subplots()

sns.boxplot(x='Condition', y='Value', data=df_boxplot, width=0.5)
sns.stripplot(x='Condition', y='Value', data=df_boxplot, color='black', linewidth=1, jitter=0)

sns.lineplot(
    data=df_boxplot, x='Condition',y='Value', units='SubID',
    color='grey', estimator=None, 
    linestyle=':', lw =  1
)

# Set labels and title
ax.set_xticklabels(['preDBS','Pre-Subharm','Subharm','postDBS'])
plt.ylabel('Peak Frequency [Hz]')
plt.xlabel('')

# Show the plot
plt.show()



In [199]:
plt.savefig(os.path.join(
    project_path, 'results',
    'PeakFrequencies'
),
dpi = 200)

plt.savefig(os.path.join(
    project_path, 'results',
    'PeakFrequencies.pdf'
))